# Титаник - Машинное обучение после катастрофы

В данном блокноте нас будет интересовать датасет [Titanic - Machine Learning from Disaster](https://www.kaggle.com/c/titanic/data).


Обучающий набор следует использовать для построения моделей машинного обучения. Для обучающего набора мы предоставляем результат (также известный как "основная истина") для каждого пассажира. Ваша модель будет основана на таких "особенностях", как пол и класс пассажиров. Вы также можете использовать разработку функций для создания новых функций.

Тестовый набор следует использовать, чтобы увидеть, насколько хорошо ваша модель работает с невидимыми данными. Для тестового набора мы не предоставляем основную истину для каждого пассажира. Ваша задача - предсказать эти результаты. Для каждого пассажира в тестовом наборе используйте обученную модель, чтобы предсказать, выжил ли он после крушения "Титаника".


In [ ]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

In [ ]:
import tensorflow as tf

import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams

In [ ]:
%matplotlib inline
rcParams['figure.figsize'] = 10,8
sns.set(style='whitegrid', palette='muted',
        rc={'figure.figsize': (15,10)})

In [ ]:
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

## Загрузка датасета с Kaggle

В ячейке ниже загружаем kaggle.json для вашей среды выполнения Colab, полученный на https://www.kaggle.com/.

In [ ]:
!pip install kaggle

В ячейке ниже загружаем kaggle.json для вашей среды выполнения Colab, полученный на https://www.kaggle.com/.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Создаем папку, в которую выгрузим датасет:

In [ ]:
!cd /content/
!mkdir dataset
!cd /content/dataset

Загрузим интересующий нас датасет

In [ ]:
!kaggle competitions download -c titanic -p /content/dataset


In [ ]:
!unzip  /content/dataset/titanic.zip -d /content/dataset

Проверяем наличие:

In [ ]:
!ls /content/dataset/ -lsa

## Разворачивание датасета

In [ ]:
# Load data as Pandas dataframe
train = pd.read_csv('/content/dataset/test.csv')
test = pd.read_csv('/content/dataset/train.csv')
data = pd.concat([train, test], axis=0, sort=True)

## Первичный анализ

In [ ]:
data.head()

In [ ]:
def display_all(data):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(data)
        
display_all(data.describe(include='all').T)

In [ ]:
data.dtypes

In [ ]:
data.nunique() # количество уникальных значений для каждого столбца 

In [ ]:
data.isna().sum()

## Кодирование категориальных признаков

In [ ]:
sns.countplot(x='Sex', data=data, palette='hls', hue='Survived')
plt.xticks(rotation=45)
plt.show()

Мы воспользуемся методом [`pandas.DataFrame.astype`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html) для преобразования в категориальный тип, а затем возьмем коды полученных объектов.

In [ ]:
# convert to cateogry dtype
data['Sex'] = data['Sex'].astype('category')
# convert to category codes
data['Sex'] = data['Sex'].cat.codes

In [ ]:
data.head()

In [ ]:
data['Title'] = data['Name'].str.split(',',1).str[1].str.split('.',1).str[0]

Выбрасываем переменные, которые мы не будем использовать

In [ ]:
data.drop(['Cabin', 'Name', 'Ticket', 'PassengerId'], axis=1, inplace=True)

Воспользуемся [`pandas.get_dummies`](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) для получение индикаторных переменных

In [ ]:
# подмножество всех категориальных переменных, которые должны быть закодированы
categorical = ['Embarked', 'Title']

for var in categorical:
    data = pd.concat([data, 
                    pd.get_dummies(data[var], prefix=var)], axis=1)
    del data[var]

In [ ]:
data.isna().sum()

## Импьютация пропущенных данных



Воспользуемся [`sklearn.impute.SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer).

In [ ]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
imp_num = SimpleImputer(missing_values=np.nan, strategy='mean')
data['Age']  = imp_num.fit_transform(data[['Age']])
data['Fare']  = imp_num.fit_transform(data[['Fare']])

## Масштабирование непрерывных переменных



Идея [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler) заключается в том, что он преобразует ваши данные так, что их распределение будет иметь среднее значение `0` и стандартное отклонение `1`.


In [ ]:
continuous = ['Age', 'Fare', 'Parch', 'Pclass', 'SibSp']

scaler = StandardScaler()

for var in continuous:
    data[var] = data[var].astype('float64')
    data[var] = scaler.fit_transform(data[var].values.reshape(-1, 1))

In [ ]:
display_all(data.describe(include='all').T)

In [ ]:
data.head()

In [ ]:
data.isna().sum()

## Обучение нейронных сетей



Теперь все, что осталось, - это передать наши данные, которые были очищены, закодированы и масштабированы в нашу нейронную сеть.

Но сначала нам нужно разделить DataFrame обратно на обучающие и тестовые наборы.

In [ ]:
X_train = data[pd.notnull(data['Survived'])].drop(['Survived'], axis=1)
y_train = data[pd.notnull(data['Survived'])]['Survived']
X_test = data[pd.isnull(data['Survived'])].drop(['Survived'], axis=1)

In [ ]:
print(f'{X_train.shape}, {y_train.shape}, {X_test.shape}')

In [ ]:
def create_model(lyrs=[8], act='linear', opt='adam', dr=0.0):
    
    model = tf.keras.Sequential()
    
    # create first hidden layer
    model.add(tf.keras.layers.Dense(lyrs[0], input_dim=X_train.shape[1], 
                                    activation=act))
    
    # create additional hidden layers
    for i in range(1,len(lyrs)):
        model.add(tf.keras.layers.Dense(lyrs[i], activation=act))
    
    # add dropout, default is none
    model.add(tf.keras.layers.Dropout(dr))
    
    # create output layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # output layer
    
    model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['acc'])
    
    return model

In [ ]:
model = create_model()
print(model.summary())

In [ ]:
history = model.fit(X_train,
                    y_train, 
                    epochs=100, 
                    batch_size=32,
                    validation_split=0.2,
                    verbose=0)

In [ ]:
history_dict = history.history
history_dict.keys()

## Оценка результатов

In [ ]:
def print_summarize_history(history):
  # summarize history for accuracy
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()

In [ ]:
print_summarize_history(history)

## Поиск по сетке

## Вычисление оптимального размера батча и эпохи



Из приведенного выше графика видно, что мы, возможно, слишком долго тренируем нашу сеть. Давайте воспользуемся поиском по сетке, чтобы узнать, каковы оптимальные значения для `batch_size` и `epochs`. Для этого воспользуемся оболочкой [Scikit-Learn API](https://keras.io/scikit-learn-api/) `KerasClassifier`. Поиск по сетке с перекрестной проверкой ([`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV)) - это грубая сила при поиске лучших гиперпараметров для конкретного набора данных и модели. 


In [ ]:
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [16, 32, 64]
epochs = [50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)

# search the grid
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid,
                    cv=3,
                    verbose=2)  # include n_jobs=-1 if you are using CPU

grid_result = grid.fit(X_train, y_train)


Сделаем какие-то выводы:

In [ ]:
def make_some_conclusion(grid_result):
  # summarize results
  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
  means = grid_result.cv_results_['mean_test_score']
  stds = grid_result.cv_results_['std_test_score']
  params = grid_result.cv_results_['params']
  for mean, stdev, param in zip(means, stds, params):
      print("%f (%f) with: %r" % (mean, stdev, param))

make_some_conclusion(grid_result)

## Поиск алгоритма оптимизации

In [ ]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=32, verbose=0)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Nadam']
param_grid = dict(opt=optimizer)

# search the grid
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=2)
grid_result = grid.fit(X_train, y_train)

Сделаем какие-то выводы:

In [ ]:
make_some_conclusion(grid_result)

## Поиск оптимального количества скрытых слоев

In [ ]:
# create model

model = KerasClassifier(build_fn=create_model, 
                        epochs=50, batch_size=32, verbose=0)

# define the grid search parameters
layers = [(8),(10),(10,5),(12,6),(12,8,4)]
param_grid = dict(lyrs=layers)

# search the grid
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=2)
grid_result = grid.fit(X_train, y_train)

Снова сделаем какие-то выводы:

In [ ]:
make_some_conclusion(grid_result)

## Dropout

In [ ]:
# create model
model = KerasClassifier(build_fn=create_model, 
                        epochs=50, batch_size=32, verbose=0)

# define the grid search parameters
drops = [0.0, 0.01, 0.05, 0.1, 0.2, 0.5]
param_grid = dict(dr=drops)
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=2)
grid_result = grid.fit(X_train, y_train)

Снова сделаем какие-то выводы:

In [ ]:
make_some_conclusion(grid_result)

## Дорабатываем модель

In [ ]:
model = create_model(opt='Nadam', lyrs=[12, 6], dr=0.0)

print(model.summary())

In [ ]:
# train model on full train set, with 80/20 CV split
training = model.fit(X_train, y_train, epochs=50, batch_size=16, 
                     validation_split=0.2, verbose=0)

In [ ]:
print_summarize_history(training)